In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Homework 3 — Spam Classification Experiments\n",
    "\n",
    "This notebook performs full data preprocessing, model training, evaluation,\n",
    "and visualization required for the assignment.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.feature_extraction.text import TfidfVectorizer\n",
    "from sklearn.pipeline import Pipeline\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.naive_bayes import MultinomialNB\n",
    "from sklearn.svm import LinearSVC\n",
    "from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc\n",
    "from src.preprocessing import clean_text, preprocess_df"
   ]
  },
  {
   "cell_type": "markdown",
   "source": [
    "## Load Dataset"
   ],
   "metadata": {}
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "outputs": [],
   "source": [
    "df = pd.read_csv(\"../Chapter03/datasets/sms_spam_no_header.csv\", header=None, names=[\"label\", \"text\"])\n",
    "df['label'] = df['label'].map({\"ham\":0, \"spam\":1})\n",
    "df = preprocess_df(df)\n",
    "df.head()"
   ],
   "metadata": {}
  },
  {
   "cell_type": "markdown",
   "source": [
    "## Train-Test Split"
   ],
   "metadata": {}
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    df['text_clean'], df['label'], test_size=0.2, stratify=df['label'], random_state=42\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "source": [
    "## Define Models"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "outputs": [],
   "source": [
    "models = {\n",
    "    \"Logistic Regression\": LogisticRegression(max_iter=2000),\n",
    "    \"Naive Bayes\": MultinomialNB(),\n",
    "    \"Linear SVM\": LinearSVC()\n",
    "}\n",
    "\n",
    "pipelines = {}\n",
    "\n",
    "for name, clf in models.items():\n",
    "    pipelines[name] = Pipeline([\n",
    "        (\"tfidf\", TfidfVectorizer(ngram_range=(1,2))),\n",
    "        (\"clf\", clf)\n",
    "    ])"
   ]
  },
  {
   "cell_type": "markdown",
   "source": [
    "## Train Models"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "outputs": [],
   "source": [
    "results = {}\n",
    "\n",
    "for name, pipe in pipelines.items():\n",
    "    pipe.fit(X_train, y_train)\n",
    "    y_pred = pipe.predict(X_test)\n",
    "    results[name] = {\n",
    "        \"y_pred\": y_pred,\n",
    "        \"pipeline\": pipe,\n",
    "        \"report\": classification_report(y_test, y_pred, output_dict=True)\n",
    "    }\n",
    "\n",
    "results"
   ]
  },
  {
   "cell_type": "markdown",
   "source": [
    "## Compare Accuracy / Precision / Recall / F1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "outputs": [],
   "source": [
    "summary = []\n",
    "for name, res in results.items():\n",
    "    rep = res['report']\n",
    "    summary.append([\n",
    "        name,\n",
    "        rep['weighted avg']['precision'],\n",
    "        rep['weighted avg']['recall'],\n",
    "        rep['weighted avg']['f1-score']\n",
    "    ])\n",
    "\n",
    "df_summary = pd.DataFrame(summary, columns=[\"Model\", \"Precision\", \"Recall\", \"F1\"])\n",
    "df_summary"
   ]
  },
  {
   "cell_type": "markdown",
   "source": [
    "## Confusion Matrix"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "outputs": [],
   "source": [
    "plt.figure(figsize=(15,4))\n",
    "for i, (name, res) in enumerate(results.items()):\n",
    "    plt.subplot(1,3,i+1)\n",
    "    cm = confusion_matrix(y_test, res['y_pred'])\n",
    "    sns.heatmap(cm, annot=True, fmt='d', cmap=\"Blues\")\n",
    "    plt.title(name)\n",
    "    plt.xlabel(\"Predicted\")\n",
    "    plt.ylabel(\"True\")\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "source": [
    "## ROC Curve (only for logistic regression)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "outputs": [],
   "source": [
    "logreg = pipelines['Logistic Regression']\n",
    "y_score = logreg.decision_function(X_test)\n",
    "fpr, tpr, _ = roc_curve(y_test, y_score)\n",
    "roc_auc = auc(fpr, tpr)\n",
    "\n",
    "plt.figure(figsize=(6,5))\n",
    "plt.plot(fpr, tpr, label=f\"AUC = {roc_auc:.3f}\")\n",
    "plt.plot([0,1],[0,1],'--')\n",
    "plt.xlabel('False Positive Rate')\n",
    "plt.ylabel('True Positive Rate')\n",
    "plt.title('ROC Curve — Logistic Regression')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined